# 유동인구 데이터 불러오기
- https://data.busan.go.kr/bdip/literacy/selectDataUtilizationCasesDetail.do?utlzCaseClsfNm=big_data_cl_04&utlzCaseId=164&pageNo=1
- 위 사이트에서 관리자도구 (F12) 켜고 원하는 데이터 요청
- Network에서 payload 확인해서 형식에 맞게 보내고, response 확인하여 형식에 맞게 가져옴
- Headers > General 에서 요청 url 등등 가져오고, Response Headers에서 Cookie, dashboard-token 등등 확인하여 불러옴
  - Cookie, token은 현재 로그인에 따라 계속 달라질 수 있기 때문에 세션 끌때마다 새로 넣어줄 필요가 있을듯

In [1]:
import pandas as pd

In [ ]:
import requests
import json

url = "https://data.busan.go.kr/bix5/api/datasources/49bc7b6cfcf86b8aa093f73240935dd2"

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Content-Type": "application/json",
    "Cookie": "JSESSIONID=5D5983DD2845941861403D08067916CC; _ga=GA1.1.565642539.1747061882; bdipContentsCookie=...worker2; ...",
    "dashboard-token": "F10E63F1318C13DD8CEC273EE9C31077C19ACF8B95C1C08337E708B39A836245D1CCFA41A02EF3907E8F18639A17DE1563F3E2D66D33FF34AEE1585C063C339F",
    "Host": "data.busan.go.kr",
    "Origin": "https://data.busan.go.kr",
    "Referer": "https://data.busan.go.kr/bix5/shares/4d58614ce1c07a11be4f6fcd7007c3b9",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"
}

payload = {
    "year": ["2024"],
    "month": ["05"],
    "guCd": [],
    "sex": []
}

res = requests.post(url, headers=headers, data=json.dumps(payload))

if res.status_code == 200:
    data = res.json()
    print(data)
else:
    print("요청 실패:", res.status_code)


{'status': 200, 'dataset': [{'지역코드': '2629000000', '지역': '남구', '생활인구': 268608}, {'지역코드': '2650000000', '지역': '수영구', '생활인구': 179151}, {'지역코드': '2623000000', '지역': '부산진구', '생활인구': 390842}, {'지역코드': '2614000000', '지역': '서구', '생활인구': 115037}, {'지역코드': '2671000000', '지역': '기장군', '생활인구': 188085}, {'지역코드': '2632000000', '지역': '북구', '생활인구': 272280}, {'지역코드': '2647000000', '지역': '연제구', '생활인구': 203363}, {'지역코드': '2653000000', '지역': '사상구', '생활인구': 244816}, {'지역코드': '2638000000', '지역': '사하구', '생활인구': 317445}, {'지역코드': '2620000000', '지역': '영도구', '생활인구': 123922}, {'지역코드': '2644000000', '지역': '강서구', '생활인구': 188308}, {'지역코드': '2635000000', '지역': '해운대구', '생활인구': 433153}, {'지역코드': '2641000000', '지역': '금정구', '생활인구': 248600}, {'지역코드': '2611000000', '지역': '중구', '생활인구': 68878}, {'지역코드': '2626000000', '지역': '동래구', '생활인구': 265722}, {'지역코드': '2617000000', '지역': '동구', '생활인구': 103594}], 'message': ' '}


# 각 지역별 성별에 따른 유동인구수 월별

In [ ]:
import requests
import pandas as pd
import json
import time

headers = {
    "Content-Type": "application/json",
    "Referer": "https://data.busan.go.kr/bix5/shares/4d58614ce1c07a11be4f6fcd7007c3b9",
    "dashboard-token": "F10E63F1318C13DD8CEC273EE9C31077C19ACF8B95C1C08337E708B39A836245D1CCFA41A02EF3907E8F18639A17DE1563F3E2D66D33FF34AEE1585C063C339F",
    "Cookie": "JSESSIONID=5D5983DD2845941861403D08067916CC; _ga=GA1.1.565642539.1747061882; bdipContentsCookie=B5962B2CD461074690786B40E934EF9C.worker2; _ga_4HTMW67C1X=GS2.1.s1747065372$o2$g0$t1747065372$j0$l0$h0; _ga_ETGLWH4KN8=GS2.1.s1747913927$o6$g0$t1747913927$j0$l0$h0; searchStyle=card; bdipLogInCookie=B5962B2CD461074690786B40E934EF9C.worker2; egovExpireSessionTime=1749275808565",
}


url = "https://data.busan.go.kr/bix5/api/datasources/49bc7b6cfcf86b8aa093f73240935dd2"

# ✅ 요청 함수 (성별 설정 가능)
def fetch_month_data(year, month, sex=[]):
    payload = {
        "year": [str(year)],
        "month": [f"{int(month):02d}"],
        "guCd": [],
        "sex": sex  # [] 전체, ['m'], ['f']
    }
    res = requests.post(url, headers=headers, data=json.dumps(payload))
    if res.status_code == 200:
        dataset = res.json().get('dataset', [])
        for row in dataset:
            row['월'] = f"{year}-{int(month):02d}"
        return dataset
    else:
        print(f"⚠️ {year}-{month} 요청 실패: {res.status_code}")
        return []

# ✅ 성별별 데이터 요청 및 병합
def collect_merged_data(years):
    all_data = []

    for year in years:
        for month in range(5, 11):
            time.sleep(0.2)

            male = fetch_month_data(year, month, ['m'])
            female = fetch_month_data(year, month, ['f'])
            total = fetch_month_data(year, month, [])  # 전체

            df_m = pd.DataFrame(male)[['월', '지역', '지역코드', '생활인구']].rename(columns={'생활인구': '생활인구_남'})
            df_f = pd.DataFrame(female)[['월', '지역', '지역코드', '생활인구']].rename(columns={'생활인구': '생활인구_여'})
            df_t = pd.DataFrame(total)[['월', '지역', '지역코드', '생활인구']].rename(columns={'생활인구': '생활인구_전체'})

            # 3개 merge
            df_merge = df_t.merge(df_m, on=['월', '지역', '지역코드'], how='left') \
                           .merge(df_f, on=['월', '지역', '지역코드'], how='left')

            all_data.append(df_merge)

    return pd.concat(all_data, ignore_index=True)

# ✅ 수집 및 저장
df_2020_2023 = collect_merged_data([2020, 2021, 2022, 2023])
df_2024 = collect_merged_data([2024])

# ✅ 엑셀로 저장
df_2020_2023.to_excel("생활인구_2020_2023_성별포함.xlsx", index=False)
df_2024.to_excel("생활인구_2024_성별포함.xlsx", index=False)


# 각 지역별 노년층 유동인구

In [ ]:
import requests
import json

url = "https://data.busan.go.kr/bix5/api/datasources/4a50b4b91ba9f924bc98acc72b235326"
headers = {
    "Content-Type": "application/json",
    "dashboard-token": "F10E63F1318C13DD8CEC273EE9C31077E1E7A0D3DA3B44E7E9615AD26E3FA6DC8607B91DC9D9CF0AF601EC5C3C641E95F7266C8A75FEB127BE5688375998EEA8",
    "Referer": "https://data.busan.go.kr/bix5/shares/430d74fc10e4d4a0ae231de8a7560c72",
    "Cookie": "JSESSIONID=5D5983DD2845941861403D08067916CC; _ga=GA1.1.565642539.1747061882; bdipContentsCookie=B5962B2CD461074690786B40E934EF9C.worker2; _ga_4HTMW67C1X=GS2.1.s1747065372$o2$g0$t1747065372$j0$l0$h0; _ga_ETGLWH4KN8=GS2.1.s1747913927$o6$g0$t1747913927$j0$l0$h0; searchStyle=card; bdipLogInCookie=B5962B2CD461074690786B40E934EF9C.worker2; egovExpireSessionTime=1749275808565",
}
payload = {
    "year": ["2020"],
    "month": ["05"],
    "guCd": [],
    "ages": ["60대", "70대"]
}

res = requests.post(url, headers=headers, data=json.dumps(payload))
print("응답 코드:", res.status_code)
print("응답 내용:", res.json())


응답 코드: 200
응답 내용: {'status': 200, 'dataset': [{'행정코드': '2644000000', '구군': '강서구', '생활인구': 30948}, {'행정코드': '2641000000', '구군': '금정구', '생활인구': 64689}, {'행정코드': '2671000000', '구군': '기장군', '생활인구': 34023}, {'행정코드': '2629000000', '구군': '남구', '생활인구': 62308}, {'행정코드': '2617000000', '구군': '동구', '생활인구': 30854}, {'행정코드': '2626000000', '구군': '동래구', '생활인구': 64465}, {'행정코드': '2623000000', '구군': '부산진구', '생활인구': 90644}, {'행정코드': '2632000000', '구군': '북구', '생활인구': 57072}, {'행정코드': '2653000000', '구군': '사상구', '생활인구': 51369}, {'행정코드': '2638000000', '구군': '사하구', '생활인구': 69622}, {'행정코드': '2614000000', '구군': '서구', '생활인구': 34806}, {'행정코드': '2650000000', '구군': '수영구', '생활인구': 46395}, {'행정코드': '2647000000', '구군': '연제구', '생활인구': 47669}, {'행정코드': '2620000000', '구군': '영도구', '생활인구': 36382}, {'행정코드': '2611000000', '구군': '중구', '생활인구': 17120}, {'행정코드': '2635000000', '구군': '해운대구', '생활인구': 90064}], 'message': ' '}


In [ ]:
import requests
import pandas as pd
import json
import time

# ✅ 설정: 요청에 필요한 헤더
headers = {
    "Content-Type": "application/json",
    "Referer": "https://data.busan.go.kr/bix5/shares/430d74fc10e4d4a0ae231de8a7560c72",
    "dashboard-token": "F10E63F1318C13DD8CEC273EE9C31077E1E7A0D3DA3B44E7E9615AD26E3FA6DC8607B91DC9D9CF0AF601EC5C3C641E95F7266C8A75FEB127BE5688375998EEA8",
    "Cookie": "JSESSIONID=5D5983DD2845941861403D08067916CC; _ga=GA1.1.565642539.1747061882; bdipContentsCookie=B5962B2CD461074690786B40E934EF9C.worker2; _ga_4HTMW67C1X=GS2.1.s1747065372$o2$g0$t1747065372$j0$l0$h0; _ga_ETGLWH4KN8=GS2.1.s1747913927$o6$g0$t1747913927$j0$l0$h0; searchStyle=card; bdipLogInCookie=B5962B2CD461074690786B40E934EF9C.worker2; egovExpireSessionTime=1749275808565",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"
}

# ✅ 요청 URL (노년층 생활인구용)
url = "https://data.busan.go.kr/bix5/api/datasources/4a50b4b91ba9f924bc98acc72b235326"

# ✅ 노년층 수집 함수 (구군 전체를 한 번에 요청)
def fetch_population_all_gu(year, month, ages=["60대", "70대"]):
    payload = {
        "year": [str(year)],
        "month": [f"{int(month):02d}"],
        "guCd": [],
        "ages": ages
    }

    try:
        res = requests.post(url, headers=headers, data=json.dumps(payload), timeout=10)
        if res.status_code == 200:
            dataset = res.json().get('dataset', [])
            result = []
            for row in dataset:
                result.append({
                    '월': f"{year}-{int(month):02d}",
                    '지역': row['구군'],
                    '지역코드': row['행정코드'],
                    '노년층_생활인구': int(row['생활인구'])
                })
            return result
        else:
            print(f"❌ 응답 실패 {year}-{month}: {res.status_code}")
            return []
    except Exception as e:
      print(f"🚨 예외 발생: {repr(e)}")  # ✅ 안전하게 출력
      return []

# ✅ 연도별 수집 함수
def collect_all_years(years):
    all_data = []
    for year in years:
        for month in range(5, 11):  # 5~10월
            print(f"🔄 수집 중: {year}-{month:02d}")
            all_data.extend(fetch_population_all_gu(year, month))
            time.sleep(0.3)
    return pd.DataFrame(all_data)

# ✅ 실행 및 엑셀 저장
df_2020_2023 = collect_all_years([2020, 2021, 2022, 2023])
df_2024 = collect_all_years([2024])

df_2020_2023.to_excel("노년층생활인구_2020_2023_통합형.xlsx", index=False)
df_2024.to_excel("노년층생활인구_2024_통합형.xlsx", index=False)


🔄 수집 중: 2020-05
🔄 수집 중: 2020-06
🔄 수집 중: 2020-07
🔄 수집 중: 2020-08
🔄 수집 중: 2020-09
🔄 수집 중: 2020-10
🔄 수집 중: 2021-05
🔄 수집 중: 2021-06
🔄 수집 중: 2021-07
🔄 수집 중: 2021-08
🔄 수집 중: 2021-09
🔄 수집 중: 2021-10
🔄 수집 중: 2022-05
🔄 수집 중: 2022-06
🔄 수집 중: 2022-07
🔄 수집 중: 2022-08
🔄 수집 중: 2022-09
🔄 수집 중: 2022-10
🔄 수집 중: 2023-05
🔄 수집 중: 2023-06
🔄 수집 중: 2023-07
🔄 수집 중: 2023-08
🔄 수집 중: 2023-09
🔄 수집 중: 2023-10
🔄 수집 중: 2024-05
🔄 수집 중: 2024-06
🔄 수집 중: 2024-07
🔄 수집 중: 2024-08
🔄 수집 중: 2024-09
🔄 수집 중: 2024-10


In [24]:
import pandas as pd

# [1] 엑셀 파일 불러오기
df_ac23 = pd.read_excel('/content/drive/MyDrive/기상청/생활인구_2020_2023_성별포함.xlsx')
df_ac24 = pd.read_excel('/content/drive/MyDrive/기상청/생활인구_2024_성별포함.xlsx')
df_se23 = pd.read_excel('/content/drive/MyDrive/기상청/노년층생활인구_2020_2023_통합형.xlsx')
df_se24 = pd.read_excel('/content/drive/MyDrive/기상청/노년층생활인구_2024_통합형.xlsx')
df_bu = pd.read_excel('/content/drive/MyDrive/기상청/부산광역시_구별_주민등록세대수(월별)_KOSIS.xlsx')

# [2] 주민등록세대수 파일 전처리
df_bu = df_bu.rename(columns={df_bu.columns[0]: '지역'})
df_bu = df_bu[df_bu['지역'] != '부산광역시']  # 부산광역시 전체 행 제거
df_bu = df_bu.melt(id_vars='지역', var_name='월', value_name='세대수')
df_bu['월'] = df_bu['월'].str.replace('.', '-', regex=False).str.strip()
df_bu['지역'] = df_bu['지역'].str.strip()

# [3] 지역코드 매핑 (지역명 기준)
지역코드_매핑 = pd.concat([df_ac23[['지역', '지역코드']], df_ac24[['지역', '지역코드']]]).drop_duplicates()
지역코드_매핑['지역'] = 지역코드_매핑['지역'].str.strip()

# 지역코드 붙이기
df_bu = pd.merge(df_bu, 지역코드_매핑, on='지역', how='left')

# 누락된 지역 확인
missing = df_bu[df_bu['지역코드'].isnull()]
if not missing.empty:
    print("❗ 누락된 지역코드가 있습니다:\n", missing['지역'].unique())

# [4] 병합 함수 정의
def merge_all(df_ac, df_se, df_bu):
    df = pd.merge(df_ac, df_se, on=['월', '지역', '지역코드'], how='left')
    df = pd.merge(df, df_bu, on=['월', '지역', '지역코드'], how='left')
    return df

# [5] 데이터 병합
df_2020_2023 = merge_all(df_ac23, df_se23, df_bu)
df_2024 = merge_all(df_ac24, df_se24, df_bu)

# [6] 결과 저장

# '월'과 '지역' 기준으로 오름차순 정렬
df_2020_2023 = df_2020_2023.sort_values(by=['월', '지역']).reset_index(drop=True)
df_2024 = df_2024.sort_values(by=['월', '지역']).reset_index(drop=True)

df_2020_2023.to_csv('/content/부산광역시_인구통합본_2020_2023.csv', index=False)
df_2024.to_csv('/content/부산광역시_인구통합본_2024.csv', index=False)

print("✅ 저장 완료: 인구통합본 2020~2023, 2024")


✅ 저장 완료: 인구통합본 2020~2023, 2024


In [25]:
df_2020_2023

,월,지역,지역코드,생활인구_전체,생활인구_남,생활인구_여,노년층_생활인구,세대수
0,2020-05,강서구,2644000000,185926,104856,81070,30948,54994
1,2020-05,금정구,2641000000,278185,134269,143916,64689,107313
2,2020-05,기장군,2671000000,174411,85798,88613,34023,71661
3,2020-05,남구,2629000000,267788,130570,137218,62308,117350
4,2020-05,동구,2617000000,101796,48760,53036,30854,45755
...,...,...,...,...,...,...,...,...
379,2023-10,수영구,2650000000,174766,81488,93278,52813,88321
380,2023-10,연제구,2647000000,199056,95010,104046,55049,94650
381,2023-10,영도구,2620000000,124082,61750,62332,42831,54373
382,2023-10,중구,2611000000,67895,32980,34915,20864,23798


In [26]:
df_2024

,월,지역,지역코드,생활인구_전체,생활인구_남,생활인구_여,노년층_생활인구,세대수
0,2024-05,강서구,2644000000,188308,107954,80354,34831,62312
1,2024-05,금정구,2641000000,248600,120267,128333,64731,105706
2,2024-05,기장군,2671000000,188085,93066,95019,41882,79637
3,2024-05,남구,2629000000,268608,131786,136822,68601,119954
4,2024-05,동구,2617000000,103594,49392,54202,32520,47354
...,...,...,...,...,...,...,...,...
91,2024-10,수영구,2650000000,177269,82539,94730,52037,88139
92,2024-10,연제구,2647000000,204811,96410,108401,55669,99326
93,2024-10,영도구,2620000000,125070,62623,62447,41210,53837
94,2024-10,중구,2611000000,68245,33156,35089,20556,23621
